In [81]:
import os
import sys

import numpy as np
import pandas as pd
import torch

dir_parts = os.getcwd().split(os.path.sep)
root_index = dir_parts.index(dir_parts[-2])
root_dir = os.path.sep.join(dir_parts[:root_index + 1])
sys.path.append(root_dir + '/code/')
import data.data_utils as data_utils
import networks.deep_resesn as deep_resesn
from networks.reservoir import ReservoirConfig
from training.solver_regression import SolverRegression

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
device = 'cpu'

# Preparation

### Settings

In [83]:
DATASET_NAME = 'SinMem10'
#DATASET_NAME = 'SinMem20'
#DATASET_NAME = 'ctXORd5'
#DATASET_NAME = 'ctXORd10'

### Data

In [84]:
if DATASET_NAME in ['SinMem10', 'SinMem20']:
    washout = 200
    if DATASET_NAME == 'SinMem10':
        delay = 10
    elif DATASET_NAME == 'SinMem20':
        delay = 20
    (train_dataset, train_target), (val_dataset, val_target), (test_dataset, test_target) = data_utils.get_sinmem(data_dir=f'{root_dir}/datasets', delay=delay, washout=washout)
elif DATASET_NAME in ['ctXORd5', 'ctXORd10']:
    washout = 200
    if DATASET_NAME == 'ctXORd5':
        delay = 5
    elif DATASET_NAME == 'ctXORd10':
        delay = 10
    (train_dataset, train_target), (val_dataset, val_target), (test_dataset, test_target) = data_utils.get_ctXOR(data_dir=f'{root_dir}/datasets', delay=delay, washout=washout)

## Model

In [85]:
def get_model(hparams):
    model = deep_resesn.deep_resesn_(hparams)
    
    n_layers = len(model.reservoir)
    # Set temporal shortcuts' matrices
    for i in range(n_layers):
        if model.reservoir[i].net.ortho is not None:
            _, M = model.reservoir[i].net.ortho.shape
            model.reservoir[i].net.ortho = network_utils.init_shortcut(device, hparams['skip_option_t'], M) 
                
    return model

# Training

In [93]:
hparams = {
    'concat': False, 
    'n_layers': 5,
    'config': ReservoirConfig(in_scaling=1, rho=1.1, alpha=0.01, beta=0.0001),
    'inter_config': ReservoirConfig(in_scaling=1, rho=0.9, alpha=0.5, beta=0.01),
    'skip_option_t': 'cycle',
}
model = deep_resesn.get_deepresesn(hparams)

In [94]:
solver = SolverRegression(
    device=device,
    model=model,
    train_data=[train_dataset, train_target],
    val_data=[val_dataset, val_target],
    test_data=[test_dataset, test_target],
    washout=washout,
    reg=0,
)
train_nrmse, val_nrmse, test_nrmse = solver.train()

Train NRMSE: 0.4502, Val NRMSE: 0.4493, Test NRMSE: 0.4803


In [92]:
print(f"Val NRMSE: {val_nrmse} - Test NRMSE: {test_nrmse}")

Val NRMSE: 1.0510084590014683 - Test NRMSE: 1.0387227039928963
